In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
pd.options.display.max_rows = 999
import plotly.express as px

In [2]:
from ast import literal_eval
def clean(x):
    return literal_eval(x)

In [3]:
accre_jobs_piped = pd.read_csv('../data/accre_jobs_cleaned.csv', converters={'node_numbers': clean})

# Maggi -- 
    Question: 3 & 4   Calculation of % of actual memory used and mean % for each group

## Notes on conversion of Mn to Mc
For Mn, convert to Mc as follows: digits / (cpus/nodes)<br>
For Mc, use the digits as is<br>
usedmem is stated in Mn, but designated as M or possibly with no designation<br>
recmem can be stated in either Mc, Mn, or none<br>
Question: The documentation says that cpus is the total cpus for the job (i.e., not the cpus per node);
    however, some jobs have 1 cpu and 2 nodes, 1 cpu and 51 nodes; so, I don't understand that.<br>
Make two new columns in memory_use reqmem_mc and usedmem_mc

In [10]:
#display the data set
accre_jobs_piped.head(2)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,['cn1531'],1188000,1188028,5120.0,2748.895417
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,['cn1441'],1188000,1090256,5120.0,2799.213333


In [11]:
#look at our groupings of 'state'
accre_jobs_piped['state'].value_counts()
#we will just want to use 'COMPLETED'

COMPLETED              3718637
CANCELLED                 6064
RUNNING                   3105
FAILED                    1379
CANCELLED by 686562        681
OUT_OF_MEMORY               94
TIMEOUT                     80
CANCELLED by 505355         63
CANCELLED by 855431         17
CANCELLED by 782611         13
CANCELLED by 397600          7
CANCELLED by 0               5
CANCELLED by 486541          4
CANCELLED by 200557          4
CANCELLED by 199766          4
CANCELLED by 454080          4
CANCELLED by 649319          3
CANCELLED by 9202            3
CANCELLED by 9201            3
CANCELLED by 483348          3
CANCELLED by 895426          2
CANCELLED by 791651          2
CANCELLED by 199066          2
CANCELLED by 666860          2
CANCELLED by 546080          2
CANCELLED by 515423          2
CANCELLED by 693461          2
CANCELLED by 90423           2
CANCELLED by 124006          2
CANCELLED by 649321          1
CANCELLED by 781109          1
CANCELLED by 782535          1
CANCELLE

In [12]:
#look at our groupings of 'partition'
accre_jobs_piped['partition'].value_counts()
#we will just want to use 'production'

production    3311788
nogpfs         327652
pascal          48004
turing          39406
maxwell          3348
Name: partition, dtype: int64

In [13]:
#look at 'exitcode' = 0:0 in 'production' by 'state'
accre_jobs_piped[(accre_jobs_piped['exitcode']=='0:0') & (accre_jobs_piped['partition']=='production')]['state'].value_counts()
#definitely limit to completed based on the 'cancelled*' and 'running' entries

COMPLETED              3302522
CANCELLED                 4449
RUNNING                   2859
CANCELLED by 686562        680
TIMEOUT                     79
CANCELLED by 9201            2
Name: state, dtype: int64

In [14]:
#pull out the initial slice of data for memory usage analysis
memory_use = accre_jobs_piped[(accre_jobs_piped['partition']=='production') & (accre_jobs_piped['exitcode']=='0:0')
                              & (accre_jobs_piped['state']=='COMPLETED')]

In [15]:
memory_use.head(2)

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,['cn1531'],1188000,1188028,5120.0,2748.895417
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,['cn1441'],1188000,1090256,5120.0,2799.213333


In [16]:
#examples where reqmem in Mc and nodes > 1
memory_use[(memory_use['reqmem'].str.contains('Mc')) & (memory_use['nodes']>1)][:10]

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc
13240,17105760,summer,buddy,20480Mc,6453.04M,04:40:00,00:00:56,2,1,production,0:0,COMPLETED,"['cn1434', 'cn1435']",16800,56,20480.0,12906.08000
13257,17105819,summer,buddy,20480Mc,7587.56M,04:40:00,00:11:43,2,1,production,0:0,COMPLETED,"['cn1434', 'cn1435']",16800,703,20480.0,15175.12000
13342,17105966,summer,buddy,20480Mc,7587.15M,04:40:00,00:14:03,2,1,production,0:0,COMPLETED,"['cn1434', 'cn1435']",16800,843,20480.0,15174.30000
26744,17125144,summer,buddy,20480Mc,20479.96M,04:40:00,00:17:50,3,1,production,0:0,COMPLETED,"['cn1448', 'cn1449', 'cn1450']",16800,1070,20480.0,61439.88000
27044,17125676,summer,buddy,20480Mc,20463.48M,04:40:00,00:34:25,3,1,production,0:0,COMPLETED,"['cn1465', 'cn1466', 'cn1467']",16800,2065,20480.0,61390.44000
28241,17126611,summer,buddy,20480Mc,6712.42M,04:40:00,00:03:37,3,1,production,0:0,COMPLETED,"['cn1430', 'cn1431', 'cn1432']",16800,217,20480.0,20137.26000
61606,17173216,arra,needham,8192Mc,65535.89M,06:00:00,00:06:44,19,8,production,0:0,COMPLETED,"['cn1281', 'cn1283', 'cn1297', 'cn1298', 'cn13...",21600,404,8192.0,155647.73875
118478,17255274,belle,osbaldo,8192Mc,0,1-00:00:00,00:01:15,4,12,production,0:0,COMPLETED,"['cn1293', 'cn1302', 'cn1355', 'cn1356']",86400,75,8192.0,0.00000
160741,17324222,crowns,tena,5120Mc,9651.66M,2-00:00:00,16:47:47,37,1,production,0:0,COMPLETED,"['cn1094', 'cn1125', 'cn1266', 'cn1277', 'cn12...",172800,60467,5120.0,357111.42000
164606,17329000,round,shenna,8000Mc,0,10:00:00,00:00:12,2,1,production,0:0,COMPLETED,"['cn1291', 'cn1292']",36000,12,8000.0,0.00000


In [17]:
#it looks like there are some jobs with 0 usedmem, does that make sense?
memory_use['usedmem_mc'].value_counts()
#no it does not, these don't seem valid for evaluating efficiency

0.0000       537283
1.5600        31241
0.1975        30085
1.5575        29904
6.2300        16918
              ...  
3493.0900         1
1416.2400         1
1393.7020         1
2559.6200         1
4122.6600         1
Name: usedmem_mc, Length: 884478, dtype: int64

In [21]:
#here's what the data looks like
memory_use[memory_use['usedmem_mc'] ==0]['used_total_seconds'].describe().apply(lambda x: format(x, 'f'))
#most are not long running jobs, but some in the top 25% are

count     537283.000000
mean          37.988003
std         2058.355733
min            0.000000
25%            7.000000
50%           12.000000
75%           20.000000
max      1013546.000000
Name: used_total_seconds, dtype: object

In [22]:
#since we'll be calculating ratios to look at efficiency, we don't want any rows where reqmem or usedmem is 0
#also remove a few stray rows where cpus = 0
memory_use = memory_use[(memory_use['reqmem_mc']>0) & (memory_use['usedmem_mc']>0) & (memory_use['cpus']>0)]

In [23]:
#some jobs have fewer cpus than nodes, which doesn't make sense
check_cpus = memory_use[memory_use['cpus']<memory_use['nodes']][['account','cpus','nodes','reqmem', 'usedmem', 'reqmem_mc','usedmem_mc' ]]

In [24]:
#here's what that data looks like
check_cpus
#it's probably not valid for efficiency ratios

,account,cpus,nodes,reqmem,usedmem,reqmem_mc,usedmem_mc
3453,winged,1,3,4096Mn,570.32M,12288.000000,1710.960000
3454,winged,1,4,4096Mn,669.61M,16384.000000,2678.440000
3457,winged,1,3,4096Mn,620.31M,12288.000000,1860.930000
3460,winged,1,4,4096Mn,629.38M,16384.000000,2517.520000
3464,winged,1,3,4096Mn,641.92M,12288.000000,1925.760000
...,...,...,...,...,...,...,...
3626272,burro,3,20,51200Mn,53.06M,341333.333333,353.733333
3629669,burro,3,20,51200Mn,235.39M,341333.333333,1569.266667
3669106,burro,3,20,51200Mn,102.50M,341333.333333,683.333333
3669107,burro,3,20,51200Mn,99.59M,341333.333333,663.933333


In [25]:
#is it limited to particular accounts?
check_cpus['account'].value_counts()
#not particularly

winged          294
chickpeas       293
angelys         196
sharlin         186
wood             62
retailer         52
burro            35
mignonette       15
crowns           13
arra             11
summer            6
peanuts           6
casaba            6
carrot            4
grisette          3
blackberries      3
crab              3
delblush          1
clara             1
cms_lowprio       1
horseradish       1
Name: account, dtype: int64

In [26]:
memory_use['account'].value_counts()

cep              464990
cms              452991
summer           372681
carrot           179941
galia            172315
cms_lowprio      116401
casaba           104134
portabella       103329
bunch             88829
orange            76909
bulk              49270
horseradish       44361
sharlin           43898
sweet             38711
key               34944
mixed             34099
retailer          32049
boysenberries     27957
wood              26226
round             23557
cmsadmin          23421
atemoyas          22084
garlic            19329
wax               17287
alkmene           15532
grisette          15176
celery            14510
winged            14108
sunburst          12876
fresh             11025
chickpeas         10486
regal              9641
large              9019
texas              8606
minnewashta        6555
clara              6324
belle              5687
lychees            5454
st                 5335
muscat             4774
bertanne           4405
seeded          

In [27]:
#remove the rows where cpus < nodes
memory_use = memory_use[memory_use['cpus']>=memory_use['nodes']]

In [28]:
memory_use['used_hours'] = (memory_use['used_total_seconds'] / 360)

In [29]:
memory_use.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc,used_hours
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,['cn1531'],1188000,1188028,5120.0,2748.895417,3300.077778
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,['cn1441'],1188000,1090256,5120.0,2799.213333,3028.488889
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,['cn1464'],1188000,1188020,5120.0,2879.660833,3300.055556
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,['cn1473'],1188000,1050632,5120.0,2721.555417,2918.422222
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,['cn1440'],1188000,1188003,5120.0,2744.837917,3300.008333


# Make a nice plot of the avg usage

In [30]:
memory_use.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc,used_hours
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,['cn1531'],1188000,1188028,5120.0,2748.895417,3300.077778
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,['cn1441'],1188000,1090256,5120.0,2799.213333,3028.488889
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,['cn1464'],1188000,1188020,5120.0,2879.660833,3300.055556
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,['cn1473'],1188000,1050632,5120.0,2721.555417,2918.422222
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,['cn1440'],1188000,1188003,5120.0,2744.837917,3300.008333


In [52]:
mem_use_by_acct = memory_use.groupby(['account'])[['reqmem_mc', 'usedmem_mc', 'used_hours']].sum().reset_index()

In [53]:
mem_use_by_acct

,account,reqmem_mc,usedmem_mc,used_hours
0,alkmene,1.793044e+07,6.048471e+06,9.883721e+04
1,almonds,3.529429e+06,6.484339e+05,4.818278e+03
2,amaranth,2.116267e+07,7.031190e+05,2.584725e+05
3,angelys,2.457600e+04,1.472480e+03,1.210675e+03
4,anise,7.154238e+06,2.499509e+06,4.034354e+04
5,antares,3.655680e+05,2.426081e+05,1.171669e+03
6,arra,5.558272e+05,3.031670e+05,1.559681e+03
7,atemoyas,2.390550e+07,1.382488e+06,1.538765e+05
8,baby,8.780000e+06,7.553961e+05,3.943189e+04
9,baking,6.439371e+06,1.873608e+06,1.474915e+05


In [69]:
#add a column for the unweighted average
mem_use_by_acct['avg_usage'] = mem_use_by_acct['usedmem_mc'] / mem_use_by_acct['reqmem_mc'] * 100

In [70]:
#total hours used, for weighting
ttl_used_hours = mem_use_by_acct['used_hours'].sum()

In [79]:
#add a column for group weighting by used_hours
mem_use_by_acct['pct_ttl_used_hrs'] = mem_use_by_acct['used_hours'] / ttl_used_hours

In [80]:
#check that the total is 1
mem_use_by_acct.pct_ttl_used_hrs.sum()

1.0

In [81]:
#add a column for weighted avg_usage
mem_use_by_acct['weighted_avg_usage'] = mem_use_by_acct['avg_usage'] * mem_use_by_acct['pct_ttl_used_hrs']

In [93]:
mem_use_by_acct

,account,reqmem_mc,usedmem_mc,used_hours,avg_usage,pct_ttl_used_hrs,weighted_avg_usage
0,alkmene,1.793044e+07,6.048471e+06,9.883721e+04,33.732975,7.127051e-04,2.404166e-02
1,almonds,3.529429e+06,6.484339e+05,4.818278e+03,18.372202,3.474412e-05,6.383259e-04
2,amaranth,2.116267e+07,7.031190e+05,2.584725e+05,3.322450,1.863819e-03,6.192446e-03
3,angelys,2.457600e+04,1.472480e+03,1.210675e+03,5.991536,8.730055e-06,5.230644e-05
4,anise,7.154238e+06,2.499509e+06,4.034354e+04,34.937457,2.909132e-04,1.016377e-02
5,antares,3.655680e+05,2.426081e+05,1.171669e+03,66.364693,8.448790e-06,5.607014e-04
6,arra,5.558272e+05,3.031670e+05,1.559681e+03,54.543394,1.124670e-05,6.134331e-04
7,atemoyas,2.390550e+07,1.382488e+06,1.538765e+05,5.783136,1.109588e-03,6.416897e-03
8,baby,8.780000e+06,7.553961e+05,3.943189e+04,8.603600,2.843394e-04,2.446342e-03
9,baking,6.439371e+06,1.873608e+06,1.474915e+05,29.096139,1.063546e-03,3.094508e-02


In [89]:
mem_use_by_acct['used_hours'].describe().apply(lambda x: format(x, 'f'))

count         131.000000
mean      1058618.019572
std       3940257.313542
min             0.325000
25%          2078.850000
50%         26677.175000
75%        261975.833333
max      30491159.675078
Name: used_hours, dtype: object

In [92]:
mem_plot_avg = mem_use_by_acct[['account','avg_usage']].set_index('account')
px_df=mem_plot_avg.sort_values(['avg_usage'])
px.bar(px_df, orientation='h', height = 2000 )

In [58]:
memory_use.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,reqmem_mc,usedmem_mc,used_hours
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,['cn1531'],1188000,1188028,5120.0,2748.895417,3300.077778
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,['cn1441'],1188000,1090256,5120.0,2799.213333,3028.488889
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,['cn1464'],1188000,1188020,5120.0,2879.660833,3300.055556
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,['cn1473'],1188000,1050632,5120.0,2721.555417,2918.422222
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,['cn1440'],1188000,1188003,5120.0,2744.837917,3300.008333


In [59]:
memory_use['used_hours'].describe().apply(lambda x: format(x, 'f'))

count    2764033.000000
mean          50.172686
std          133.541018
min            0.000000
25%            1.977778
50%            6.241667
75%           40.036111
max         4760.736111
Name: used_hours, dtype: object

In [ ]:
#slice out some short jobs
job_duration_short = memory_use[memory_use['used_hours'] <= 2]
job_duration_short.head().to_csv('../Data/short.csv')

In [41]:
#slice out some longer jobs
job_duration_long = memory_use[(memory_use['used_hours'] >4) & 
                               (memory_use['used_hours'] <8)
                              ]
job_duration_long.head().to_csv('../Data/longer.csv')

In [50]:
#slice out some really long jobs
job_duration_xlong = memory_use[(memory_use['used_hours'] > 72)
                              ]
job_duration_xlong.head().to_csv('../Data/xlong.csv')

In [ ]:
#full_df['MEM_WEIGHT'] = full_df['MEM_PERCENT_USED']*full_df['TOTAL_SEC_USED']
#full_df.groupby('ACCOUNT')['MEM_WEIGHT'].sum()/full_df.groupby('ACCOUNT')['TOTAL_SEC_USED'].sum()
#alkmene 38.351490
#almonds 46.785636
#amaranth 4.135943